# Onboarding and getting started with AMC APIs using Python 

The Amazon Marketing Cloud (AMC) API Python collection is a set of pre-configured Python code that enables you to easily onboard Amazon Ads and start exploring AMC APIs.

This collection contains the libraries, functions, and code required to get you onboarded Amazon Ads. Which means, some sections of this collection need to be executed only once to help you with your initial access and onboarding process, while the others must be executed before the start of every session.

To use this collection, ensure you have installed:
- Python (version >=3.8) with urllib3<2.0.0,>=1.25.8
- AWS CLI with botocore (required if you are using AWS Secrets Manager)

To onboard the Amazon Ads API, you must complete the following prerequisites:

- **Create a Login with Amazon application**: Requests to the Amazon Ads API are made by a client application administered by Login with Amazon.
- **Apply for permission to access Amazon Ads API**: The application form includes questions about how your business intends to use the Amazon Ads API, as well as information about the Amazon Ads API License Agreement, the Data Protection Policy, and Amazon Marketing Cloud Terms. If you are a partner, visit the Amazon Ads Partner page for information on how to apply for access to the API.
- **Assign API access to your Login with Amazon application**. This step must be completed after your application is granted permission to access Amazon Ads API along with the scopes your application can use.

You can now execute each code block to begin your onboarding journey. 

## Onboard Amazon Ads API

To initiate your onboarding journey, perform the following steps:

- [Import required libraries](#import-libraries)
- [Run functions](#run-functions-at-the-start-of-each-session)
- [Manager credentials through secrets manager (optional)](#manage-credentials-through-a-secrets-manager)
- [Create an authorization grant](#create-an-authorization-grant)
- [Understand mandatory header parameters]()

## Get started with AMC APIs
If you have already onboarded Amazon Ads API, perform the following steps before to explore AMC APIs using this collection:
- [Import required libraries](#import-libraries)
- [Run functions](#run-functions-at-the-start-of-each-session)
- [Generate access and refresh tokens](#generate-refresh-access-token-at-start-of-every-session)
- [Explore AMC APIs](#amc-apis)


### Import libraries

Libraries required to run this collection must be imported during onboarding and at the start of every session. To import libraries, execute the following code block. 

In [ ]:
## Import Libraries 
## Execute this code block before each session to import libraries
import requests
import json
import pandas as pd
import boto3
from botocore.exceptions import ClientError
from pprint import pprint
import sys

### Functions used in the collection

The following functions are used by the code blocks in this collection and must be executed while onboarding Amazon Ads API and at the start of each session. 
To initialize the functions referenced through the this collection, execute the following code block. 

In [ ]:
## Functions

## Execute this code block before each session to initialize the functions to use

## Function to retrieve credentials from secrets manager

def get_secret(secret_name, region_name):

    ## Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        ## For a list of exceptions thrown, see
        ## https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e
        
    print("Secrets retrieved")
    
    return json.loads(get_secret_value_response['SecretString'])


## Function to refresh access token

def refresh_access_token(url, Clientid, Clientsecret, refresh_token):
    refresh_body = {"grant_type": "refresh_token",
                 "client_id": Clientid,
               "client_secret": Clientsecret,
               "refresh_token": refresh_token}

    r = requests.post(url, data = refresh_body)

    if r.ok:
        access_token = r.json()['access_token']
        print("Token refreshed successfully")
        
        return access_token
    else:
        print('Response Code is: ', r)
        print('Error message is: ', r.text)
        raise Exception("Access token failed to refresh")

### Manage credentials through a secrets manager

In this example, we are using [AWS Secrets Manager](https://docs.aws.amazon.com/secretsmanager/latest/userguide/retrieving-secrets-python-sdk.html) to manage our credentials. 

> **_NOTE:_** Know that this is not an essential step and you can choose to hard-code your credentials here or use any other secrets manager. However using a secrets manager is encouraged over hardcoding.

In [ ]:
## Modify the values of the variables in this code block that the collection will refer to. Execute it at the start of each session.

## In our example, we store the Clientid, Clientsecret, returnurl, and refresh_token in the secrets manager. 
## These variables help with authenticating your calls. 
## While Clientid, Clientsecret, returnurl are constant variables, 
## you will need to update the your secrets manager with the value of refresh_token at the start of each session.    

## Set Variables
secret_name = "secrets-name-here"
region_name = "us-east-1"
    
secretDict = get_secret(secret_name = secret_name, region_name=region_name)
Clientid = secretDict['Clientid']
Clientsecret = secretDict['Clientsecret']
returnurl = secretDict['returnurl']

## Update the value of refresh token after you generate the access token and refresh tokens the first time. 
## After refresh tokens are generated initialize here. This value must be updated at the start of each session.  

#refresh_token = secretDict['refreshToken']

## Store the instance ID of your instance here. You will not need the instanceid for onboarding.
## After onboarding, uncomment the line below and update the value to store your instance id. In this example, the instanceid is 'amc-instance01'. 

instanceid = 'amc-instance01'

## Your AMC Account Id, also referred to as entityId here
## In this example, the entityId is 'ENTITY1AA1AA11AAA1'   
## if you don't know this this value use the GET operation of the `/amc/accounts` endpoint that returns a list of all the accounts your client Id has access to

#entityId = 'ENTITY1AA1AA11AAA1'

### Create an authorization grant

Only approved client applications can make calls to the Amazon Ads API on behalf of an Amazon user account with access to Amazon Ads accounts.

The relationship between client application and user account is administered by Login with Amazon (LwA). Accounts must explicitly grant authorization to the client application through LwA's Authorization Code Grant process to generate an authorization code. For more details, see [Create an authorization grant.](https://advertising.amazon.com/API/docs/en-us/guides/get-started/create-authorization-grant)

In [ ]:
## Generate an authorization grant - execute this code block only once.

#urlgrant = f'https://www.amazon.com/ap/oa?client_id={Clientid}&scope=advertising::campaign_management&advertising::audiences&response_type=code&redirect_uri={returnurl}'


print(urlgrant)

### Generate an access token for the first time

If the parameters in the above URL are valid, the page refreshes to display your client application, asking you for permission to use the application. 

Click **Allow**. You will now be redirected to the 'redirect url'. 

The address bar of your browser also displays the authorization code that you will use to generate an access token.
You will need to [generate an access token for the first time,](https://advertising.amazon.com/API/docs/en-us/guides/account-management/authorization/access-tokens) and use the access token to generate a refresh token. For subsequent sessions, user [refresh the access  access tokens.](https://advertising.amazon.com/API/docs/en-us/guides/account-management/authorization/access-tokens)

Grab the authorization code generated here, pass it as the value for the 'code' (auth_code) parameter below.
Note that auth_codes expire after 5 minutes of being generated.

In [ ]:
## Generate access token and refresh tokens - one time step 
## After your refresh token is generated through this code, you do not need once to run this code again, 
## instead use the (next) code block that uses the refresh token to generate access tokens. 
  
tokenurl = 'https://api.amazon.com/auth/o2/token'
auth_code = 'Insert authorization code generated in your redirect url'


token_body = {"grant_type": 'authorization_code',
              "client_id": 'Clientid',
            "client_secret": 'Clientsecret',
            "redirect_uri": 'returnurl',
            "code": 'auth_code'}

r = requests.post(tokenurl, data = token_body)
print(r.text)

if r.ok:
     access_token = r.json()['access_token']
     refresh_token = r.json()['refresh_token']

## Save the refresh_token in your secrets manager 

With that, you have completed the process to onboard to Amazon Ads. 
Now, ensure you have defined the mandatory header parameters.

### Mandatory header parameters

Listed below are the header parameters you will need to include to most of your API requests. Some requests may require additional parameters. See the Amazon Marketing Cloud section of the [AMC API references](https://advertising.amazon.com/API/docs/en-us/reference/api-overview) for details on each request. 

| Header | Description |
|---|---|
| Amazon-Advertising-API-ClientId | The identifier of a client associated with an Amazon Developer account. |
| Amazon-Advertising-API-MarketplaceId | The marketplace identifier for the marketplace in the request. Marketplaces are tied to the country. |
| Amazon-Advertising-API-AdvertiserId | The AMC account identifier. Your Amazon Ads account executive would have provided you this identifier when you onboarded AMC. Alternatively, access the AMC console with your user identifier and password, and grab the entity identifier value that is displayed in the URL. The alphanumeric code that is prefixed with ENTITY is your account identifier. You can also use the GET operation of the `/amc/accounts` endpoint that returns a list of all the accounts your client Id has access to. An example of an account identifier is: ENTITY1AA1AA11AAA1. |
| Authorization | Login with Amazon token in the form of Bearer {token} |


### Generate refresh access token

Once tokens have been retrieved using the authorization code, a new access token [can be retrieved at any time using the refresh token.](https://advertising.amazon.com/API/docs/en-us/guides/account-management/authorization/access-tokens#generating-an-access-token-using-a-refresh-token) 
This step must be performed at the start of each session. 

In [ ]:
## Refresh access token for each session 
## Refresh tokens are valid for one hour

access_token =  refresh_access_token(url = 'https://api.amazon.com/auth/o2/token', 
                     Clientid=Clientid,Clientsecret=Clientsecret, 
                     refresh_token=refresh_token) 


url = 'https://advertising-api.amazon.com'

## For this collection, we'll use the marketplaceId for North America

headers = {
        'Authorization': 'Bearer ' + access_token,
        'Amazon-Advertising-API-ClientId': Clientid,
        'Amazon-Advertising-API-MarketplaceId': 'ATVPDKIKX0DER'
## Your Amazon Ads account executive would have provided you this an entity identifier (entityId) when you onboarded AMC. 
## This entity id is referred to as Amazon-Advertising-API-AdvertiserId in the headers.
## Access the AMC console with your user identifier and password, and grab the entity identifier value that is displayed in the URL. 
## The alphanumeric code that is prefixed with ENTITY is your account identifier. An example of an entityId is: ENTITY1AA1AA11AAA1.
## Update this value with the entityId
        #'Amazon-Advertising-API-AdvertiserId': ENTITY1AA1AA11AAA1

       }

## AMC APIs

- [AMC account and instance administration APIs]()
- [AMC Reporting APIs - Workflow management service]()
    - Manage workflows
    - Manage workflow schedules
    - Manage workflow executions 

### AMC account and instance administration APIs

Your instances are linked to an account. While only an AMC admin can manage user access to the instances associated with the account, all users can view the details of an AMC their instance is part of.
Use the request below to list the AMC account values that you will pass as a required value for the 'Amazon-Advertising-API-AdvertiserId' header parameter, which is mandatory for all API calls. Additionally, the  response also returns the 'Amazon-Advertising-API-MarketplaceId', which is the marketplace of the account.  

In [ ]:
## List AMC account

r = requests.get(url +  "/amc/accounts", headers = headers)

print(r)
print(r.text)

if r.ok:
    ## Select which element of the list is the correct choice for your use case. 
    # Replace 'X' with the chosen integer to get a single value

    accountId = r.json()['amcAccounts'][X]['accountId']
    marketplaceId = r.json()['amcAccounts'][X]['marketplaceId']
    
    print(accountId, marketplaceId)

In [ ]:
## View the AMC instances associated with your account

instances = requests.get(url + '/amc/instances', headers = headers)
print(instances)
print(instances.json()['instances'])

### AMC Reporting APIs - Workflow management service

#### Manage workflows 

In [ ]:
## List Existing Workflow

print(instanceid)
url = 'https://advertising-api.amazon.com'

r = requests.get(url + f'/amc/reporting/{instanceid}/workflows', headers = headers)
print(r)
if r.ok:
    print(r.text[:50])
    #pprint(r.json())
else:
    print(f"Error: {r.text}")

##### AMC SQL

AMC supports most of the basic query operations Structured Query Language (SQL) offers. We refer to the SQL used in AMC as AMC SQL. For details on on the AMC SQL functions, its limitations, and guidance on optimizing AMC SQL queries, see [AMC SQL.](https://advertising.amazon.com/API/docs/en-us/guides/amazon-marketing-cloud/amc-sql/overview)  

While AMC APIs require you to construct your SQL queries as a single-line query, stripped of of tabs and new-line delimiters, before adding it to the request body, the SQL queries here do NOT need to be constructed as a single-line query. 


For example, we will be using the following SQL body:

(# This code snippet is for illustration purpose; executing this will not yield results)

```SQL

SELECT
campaign,
SUM(total_purchases) AS total_orders_9d
FROM
amazon_attributed_events_by_traffic_time
WHERE
SECONDS_BETWEEN(traffic_event_dt_utc, conversion_event_dt_utc) <= 60 * 60 * 24 * 9
GROUP BY
campaign

```

When defining your SQL body to use in the workflow template, you can choose to leave it as above. 
Or, define it as a single-line query as illustrated below:

In [ ]:
## Define SQL body that your workflow will use
sql = """
SELECT campaign, SUM(total_purchases) AS total_orders_9d FROM amazon_attributed_events_by_traffic_time WHERE SECONDS_BETWEEN(traffic_event_dt_utc, conversion_event_dt_utc) <= 60 * 60 * 24 * 9 GROUP BY campaign
"""

In [ ]:
## Create workflow template to call the SQL code defined above
body = {
        "workflowId": "workflow-name",
        "distinctUserCountColumn": "du_count",
        "filteredMetricsDiscriminatorColumn": "filtered",
        "filteredReasonColumn": "true",
        "sqlQuery": sql,
        }

r = requests.post(url + f'/amc/reporting/{instanceid}/workflows', headers = headers, data = json.dumps(body))
print(r)
print(r.text)

In [ ]:
## Delete Workflow Template
r = requests.delete(url + f'/amc/reporting/{instanceid}/workflows/workflow-name', headers = headers)
print(r)
print(r.text)

#### Manage workflow schedules

In [ ]:
workflowId = 'workflow-name'
scheduleId = 'schedule-id'

In [ ]:
## List workflow schedules
r = requests.get(url +  f'/amc/reporting/{instanceid}/schedules', headers = headers)
print(r)
print(r.text)

In [ ]:
## Create a schedule for a workflow
body = {
    'scheduleId': scheduleId,
    'workflowId': workflowId,
    'aggregationHourUtc': 8,
    'aggregationPeriod': 'Daily',
    'aggregationStartDay': 'Tuesday',
    'scheduleEnabled' : 'True'
}
r = requests.post(url +  f'/amc/reporting/{instanceid}/schedules', headers = headers, data = json.dumps(body))
print(r)
print(r.text)

In [ ]:
## Retrieve a schedule

r = requests.get(url +  f'/amc/reporting/{instanceid}/schedules/{scheduleId}', headers = headers)
print(r)
print(r.text)

In [ ]:
## Delete a schedule

r = requests.delete(url +  f'/amc/reporting/{instanceid}/schedules/{scheduleId}', headers = headers)
print(r)
print(r.text)

#### Manage workflow executions

In [ ]:
## Execute workflow using workflow ID
## For detailed descriptions of each parameter, see https://advertising.amazon.com/API/docs/en-us/amc-reporting#tag/Workflows

body = {
    "timeWindowStart": "2024-02-01T00:00:00Z",
    "timeWindowEnd": "2024-03-31T00:00:00Z",
    "timeWindowType": "EXPLICIT",
    "workflowId": "workflow-name",
    "ignoreDataGaps": "True",
    "workflowExecutionTimeoutSeconds": "86400"
    ##If using a Sandbox instance, uncomment the following:
    #"requireSyntheticData": "True"
 }

r = requests.post(url + f'/amc/reporting/{instanceid}/workflowExecutions', headers = headers, data = json.dumps(body))
print(r)
print(r.text)

if r.ok:
    workflowExecutionId = r.json()['workflowExecutionId']
    print("workflowExecutionId updated") 

In [ ]:
## Pull the workflowExecutionId from original workflow execute request 
## Overwrite value set in previous cell if needed

# workflowExecutionId = "11111111"

In [ ]:
## List all executions of a certain workflowId. 
## Example: list past executions or find a workflowId that was not stored 

r = requests.get(url + f'/amc/reporting/{instanceid}/workflowExecutions?workflowId={workflowExecutionId}', headers = headers)
print(r)
print(r.json())

In [ ]:
## Retrieve status information about the requested workflow execution.
r = requests.get(url + f'/amc/reporting/{instanceid}/workflowExecutions/{workflowExecutionId}', headers = headers)
print(r)
print(r.text)

if r.ok:
    print(f"\n Status of workflow execution is: {r.json()['status']}")

In [ ]:
## Retrieve pre-signed url for downloading workflow execution results from S3

r = requests.get(url + f'/amc/reporting/{instanceid}/workflowExecutions/{workflowExecutionId}/downloadUrls', headers = headers)
print(r)
print(r.text)

In [ ]:
## Read "Download URL" into pandas dataframe

#df = pd.read_csv(r.json()['downloadUrls'][0])
#df